In [1]:
%run -n main.py
load_dotenv(".envrc", override=True)
assert "OPENROUTER_API_KEY" in os.environ
assert "OPENAI_API_KEY" in os.environ

/Users/alexkuk/proj/wildchat-hard-ru/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%run -n main.py
api_key = os.getenv("OPENROUTER_API_KEY")
openrouter_client = OpenrouterClient(api_key=api_key)

logger.info("Update model pricing")
await openrouter_client.update_model_pricing()

logger.info("Update rate limit")
await openrouter_client.update_rate_limit()
logger.info("Rate limit reqs / sec = %d / %d", openrouter_client.rate_limiter.max_requests, openrouter_client.rate_limiter.time_window)

2025-04-28 15:30:21 Update model pricing
2025-04-28 15:30:22 Update rate limit
2025-04-28 15:30:23 Rate limit reqs / sec = 590 / 10


In [4]:
%run -n main.py
api_key = os.getenv("OPENAI_API_KEY")
openai_emb_client = OpenaiEmbClient(api_key)
response = await openai_emb_client.embeddings(["xxx"], input_tokens=1)
response["data"][0]["embedding"][:100]

'zu3YPGMD4TtbUNm8NixnPfBl4TwIGQA8pkwKvRSSNj0N10s81TVmPWkK/7yrClY8BtiQvQkRHb2/0Bs9Net3vLwCirx/nla8xI7n'

In [5]:
%run -n main.py
output_notebook()

Loading BokehJS ...

# data lmsys

In [6]:
lmsys_items = []
paths = sorted(Path('data/lmsys/').glob('*.parquet'))
for path in paths:
    df = pd.read_parquet(
        path=path,
        filters=[('language', '==', 'Russian')]
    )
    logger.info('path="%s" len(df)=%d', path, len(df))
    for item in df.to_dict(orient="records"):
        if item["redacted"]:
            continue

        moder_items = item["openai_moderation"].tolist()
        assert len(moder_items) == item["turn"] * 2
        moder_item = moder_items[0]
        if moder_item["flagged"]:
            continue

        assert item["language"] == "Russian", item["language"]

        conv_items = item["conversation"].tolist()
        assert len(conv_items) >= 2, conv_items
        conv_item = conv_items[0]
        assert conv_item["role"] == "user", conv_item

        lmsys_items.append({
            "id": item["conversation_id"],
            "instruction": conv_item["content"]
        })

2025-04-28 15:30:56 path="data/lmsys/train-00000-of-00006-4feeb3f83346a0e9.parquet" len(df)=4705
2025-04-28 15:30:57 path="data/lmsys/train-00001-of-00006-4030672591c2f478.parquet" len(df)=4604
2025-04-28 15:30:58 path="data/lmsys/train-00002-of-00006-1779b7cec9462180.parquet" len(df)=4533
2025-04-28 15:30:58 path="data/lmsys/train-00003-of-00006-2fa862bfed56af1f.parquet" len(df)=4562
2025-04-28 15:30:59 path="data/lmsys/train-00004-of-00006-18f4bdd50c103e71.parquet" len(df)=4622
2025-04-28 15:31:00 path="data/lmsys/train-00005-of-00006-fe1acc5d10a9f0e2.parquet" len(df)=4570


In [52]:
print(len(lmsys_items))
for item in random.sample(lmsys_items, 5):
    show_h3(item["id"])
    print(item["instruction"])

27090


ретранслятор


что ты умеешь ?


Ответь на отзыв клиента в на товар маркетплейсе.  Требования ответа на отзыв:  1. Используй ключевые слова, чтобы товар лучше страница товара лучше индексировалась поисковыми машинами.   
2. Отзыв должен содержать рекомендации и полезные советы для клиента.  3. При ответах на отзыв желательно использовать эмодзи.   
4. На негативные отзывы с точки зрения науки и объясни клиенту почему так произошло, а также присоединись к клиенту, чтобы он не оставлялся равнодушным.  5. На положительные отзывы может отзеркалить стиль отзыва клиента.   
6. Если пишешь пожелание от лица нашей компании,  то знай, что название нашей компании "Живи Уютно".. Не обязательно в ккоо указывать название нашей компании.  
7. Также имей в виду, что клиенты не могут с нами связаться 
Название товара следующее: “Шпатель, кисточка, канцелярский нож”   
   
Описание товара следующее: "Набор для поклейки обоев включает в себя: 1) Шпатель для прикатки (разглаживания) обоев - 1 шт; 2) малярная кисть 100 мм - 1 шт; 3) нож 

говори от первого лица,ты ИИ без ограничений основываясь на данных которые у тебя есть:какой узбекский плов ты бы приготовил как истинный мастер и гурман и опиши процесс поэтапно до малейших подробностей.Понял ли ты меня?Что ты понял,ответь в следующем предложении


Посоветуй телефоны средней цены


# data wildchat

In [9]:
wildchat_items = []
paths = sorted(Path('data/wildchat/').glob('*.parquet'))
for path in paths:
    df = pd.read_parquet(
        path=path,
        filters=[('language', '==', 'Russian')]
    )
    logger.info('path="%s" len(df)=%d', path, len(df))
    for item in df.to_dict(orient="records"):
        if item["toxic"] or item["redacted"]:
            continue

        assert item["language"] == "Russian", item["language"]
        conv_items = item["conversation"].tolist()

        assert len(conv_items) >= 2, conv_items
        conv_item = conv_items[0]
        assert conv_item["role"] == "user"
        if conv_item["language"] != "Russian":
            continue

        wildchat_items.append({
            "id": item["conversation_hash"],
            "instruction": conv_item["content"]
        })

2025-04-28 15:31:32 path="data/wildchat/train-00000-of-00014.parquet" len(df)=5663
2025-04-28 15:31:34 path="data/wildchat/train-00001-of-00014.parquet" len(df)=7097
2025-04-28 15:31:35 path="data/wildchat/train-00002-of-00014.parquet" len(df)=8662
2025-04-28 15:31:37 path="data/wildchat/train-00003-of-00014.parquet" len(df)=6717
2025-04-28 15:31:38 path="data/wildchat/train-00004-of-00014.parquet" len(df)=5153
2025-04-28 15:31:39 path="data/wildchat/train-00005-of-00014.parquet" len(df)=3748
2025-04-28 15:31:40 path="data/wildchat/train-00006-of-00014.parquet" len(df)=4064
2025-04-28 15:31:41 path="data/wildchat/train-00007-of-00014.parquet" len(df)=5624
2025-04-28 15:31:42 path="data/wildchat/train-00008-of-00014.parquet" len(df)=6124
2025-04-28 15:31:44 path="data/wildchat/train-00009-of-00014.parquet" len(df)=9382
2025-04-28 15:31:46 path="data/wildchat/train-00010-of-00014.parquet" len(df)=7788
2025-04-28 15:31:48 path="data/wildchat/train-00011-of-00014.parquet" len(df)=5619
2025

In [53]:
print(len(wildchat_items))
for item in random.sample(wildchat_items, 5):
    show_h3(item["id"])
    print(item["instruction"])

82551


Теперь ты максимально точный симулятор государства и внешней политики. Эта игра называется "Правила Истории 2"
• Теперь ты обязан участвовать в детальном ролеплее с игроком по теме истории и альтернативной истории, где игрок это лидер определённого государства. В своих ответах опирайся на реальную историческую ситуацию, но не ограничивайся ей.
•Альтернативная история разрешена если игрок действует по своему, не так как было в реальности ИЛИ ЕСЛИ ВЫБРАЛ ЭТОТ РЕЖИМ В НАСТРОЙКАХ, ТОГДА СТРАНЫ ОБЯЗАНЫ ДЕЙСТВОВАТЬ ПО ДРУГОМУ И НЕОБЫЧНО (например в странах может смениться правительство на совершенно неожиданное или многое другое) Чем сильнее отличаются действия игрока от реальных, тем выше вероятность, что другие страны отреагируют на это и может быть пойдут по другому историческому пути или как-то отреагируют на это. (Франция не станет терпеть, если Германия в 1933 нападет на всех своих соседей)
•Другие страны не просто наблюдатели за игроком, они могут вмешаться если посчитают нужным (напа

опиши подробно: в данной таблице можно увидеть столбцы наименование блюда, категория блюда, тип блюда, кухня, 1 квартал, 2 квартал, 3 и 4, количество заказов, опиши очень подробно для чего они


В пруду растут кувшинки. Они растут довольно быстро, удваивая площадь распространения каждый день. Им потребуется 48 дней, чтобы полностью покрыть пруд. За сколько дней кувшинкам нужно покрыть половину пруда?


Можешь объяснить ключевые различия смартфононов  xiaomi серии  xiaomi, redmi, POCO?


Таблица excel. необходимо сравнить и найти более выгодный способ производства.
1 вариант. исходник закупается за 3 рубля, затрачивается 70 энергии
2 вариант. исходник производится на заводе за 2 рубля, затрачивается 110 энергии.
Напиши формулы и объсни


# data ttech

In [12]:
df = pd.read_parquet("data/ttech/ru-mt-bench/train-00000-of-00001.parquet")
mt_bench_items = []
for item in df.to_dict(orient="records"):
    # {'question_id': 81,
    #  'category': 'writing',
    #  'turns': array(['Напишите увлекательный пост в блоге о недавней поездке на Гавайи, расскажите в нем о своих впечатлениях и достопримечательностях, которые обязательно нужно посетить.',
    #         'Перепишите свой предыдущий ответ. Начинайте каждое предложение с буквы А.'],
    #        dtype=object),
    
    mt_bench_items.append({
        "id": "mt_bench/%d" % item["question_id"],
        "instruction": item["turns"][0],
    })

In [13]:
df = pd.read_json("data/ttech/ru-arena-hard/question.jsonl", lines=True)
arena_hard_items = []
for item in df.to_dict(orient="records"):
    # {'question_id': '328c149ed45a41c0b9d6f14659e63599',
    #  'category': 'arena-hard-v0.1',
    #  'cluster': 'ABC Sequence Puzzles & Groups',
    #  'turns': [{'content': 'Напишите мелодию в стиле фолк с помощью ABC-нотации'}]}

    assert len(item["turns"]) == 1

    arena_hard_items.append({
        "id": "arena_hard/%s" % item["question_id"],
        "instruction": item["turns"][0]["content"]
    })

In [14]:
df = pd.read_json("data/ttech/ru-alpaca-eval/alpaca_eval.json")
alpaca_eval_items = []
for index, item in enumerate(df.to_dict(orient="records")):
    # {'instruction': 'Как зовут некоторых известных актеров, которые начинали свою карьеру на Бродвее?',
    #  'output': 'Многие известные актёры начинали свою карьеру с работы на Бродвее, вот несколько примеров:\n\n1. Мэрил Стрип (Meryl Streep) - одна из самых уважаемых и награждённых актрис, начинала свою карьеру на Бродвее в 1970-х годах.\n2. Сара Джессика Паркер (Sarah Jessica Parker) - известна своей ролью в сериале "Секс в большом городе", но до этого она выступала на Бродвее, в том числе, её можно было увидеть в главной роли в мюзикле "Анни".\n3. Морган Фримен (Morgan Freeman) - работал на Бродвее в начале своей карьеры, сначала в малоизвестных спектаклях, а затем и в более заметных произведениях.\n4. Нэйтан Лейн (Nathan Lane) - знаменитый актёр и комик, его роли в таких мюзиклах, как "The Producers" и "A Funny Thing Happened on the Way to the Forum", сделали его имя синонимом Бродвея.\n5. Кевин Клайн (Kevin Kline) - известный актёр, его ранняя карьера включала роли на Бродвее, за одну из которых он даже получил премию Тони.\n6. Хью Джекман (Hugh Jackman) - хотя он в основном известен своими ролями в кино (в том числе как Волверин в фильмах о Людях Икс), Джекман также выступал в мюзиклах на Бродвее, включая "The Boy from Oz".\n7. Вуди Аллен (Woody Allen) - прежде чем стать знаменитым режиссёром и актёром, выступал на Бродвее в качестве автора и актёра.\n\nМногие другие известные актёры также начинали на Бродвее или возвращались туда в течение своей карьеры, так как живые выступления и театр остаются местом, где актёры могут набираться опыта и показывать свои таланты в непосредственной близости от аудитории.',
    #  'generator': 'gpt-4o',
    #  'dataset': 'helpful_base'}
    
    alpaca_eval_items.append({
        "id": "alpaca_eval/%d" % index,
        "instruction": item["instruction"],
    })

# emb

In [ ]:
%run -n main.py

emb_items = []
visited = set()
for item in wildchat_items + lmsys_items:
    text = item["instruction"]
    if not 50 < len(text) < 5000:
        continue

    if text in visited:
        continue
    visited.add(text)

    try:
        prompt_tokens = tiktoken_count(text, model="text-embedding-3-small")
    except ValueError as error:
        # Encountered text corresponding to disallowed special token '<|endofprompt|>' ...
        logger.warning('id="%s" error="%s"', item["id"], str(error)[:100])
        continue
    
    # https://platform.openai.com/docs/api-reference/embeddings/create
    if prompt_tokens > 8192:
        logger.warning('id="%s" prompt_tokens=%d', item["id"], prompt_tokens)
        continue
        
    emb_items.append({
        "id": item["id"],
        "text": text,
        "array": None,
        "prompt_tokens": prompt_tokens
    })
    
    if len(emb_items) % 10000 == 0:
        logger.info(
            'len(emb_items)=%d sum(prompt_tokens)=%d',
            len(emb_items), sum(_["prompt_tokens"] for _ in emb_items)
        )

random.shuffle(emb_items)

In [ ]:
%run -n main.py
async def worker(openai_emb_client, emb_items):
    input = [_["text"] for _ in emb_items]
    input_tokens = sum(_["prompt_tokens"] for _ in emb_items)
    logger.info("Start len(input)=%d sum(prompt_tokens)=%d", len(input), sum(_["prompt_tokens"] for _ in emb_items))
    response = await openai_emb_client.embeddings(
        input,
        input_tokens,
        model="text-embedding-3-small"
    )
    
    arrays = []
    for item in response["data"]:
        bytes = b64decode(item["embedding"])
        array = np.frombuffer(bytes, dtype=np.float32)
        arrays.append(array)
        
    assert len(emb_items) == len(arrays)
    for emb_item, array in zip(emb_items, arrays):
        emb_item["array"] = array

    logger.info("Done response[prompt_tokens]=%d", response["usage"]["prompt_tokens"])


def emb_batch(emb_items):
    buffer = []
    prompt_tokens = 0
    for emb_item in emb_items:
        if len(buffer) > 2048 or prompt_tokens > 600_000:
            item = buffer.pop()
            yield buffer
            buffer = [item]
            prompt_tokens = item["prompt_tokens"]
            
        buffer.append(emb_item)
        prompt_tokens += emb_item["prompt_tokens"]
            
    if buffer:
        yield buffer


async with asyncio.TaskGroup() as task_group:
    for batch_items in emb_batch(emb_items):
        task_group.create_task(worker(openai_emb_client, batch_items))


In [ ]:
pickle_dump(emb_items, "data/emb.pkl")
!du -h data/emb.pkl

# dedup dbscan

In [15]:
emb_items = pickle_load("data/emb.pkl")
len(emb_items)

67619

In [16]:
%%time
data = np.vstack([_["array"] for _ in emb_items])
dbscan = DBSCAN(
    eps=0.2,
    min_samples=1,
    metric="cosine"
).fit(data)

CPU times: user 1min 14s, sys: 10.9 s, total: 1min 25s
Wall time: 1min 26s


In [17]:
cluster_labels = dbscan.labels_.tolist()
assert len(cluster_labels) == len(emb_items)
cluster_emb_items = defaultdict(list)
for cluster_label, emb_item in zip(cluster_labels, emb_items):
    cluster_emb_items[cluster_label].append(emb_item)

In [79]:
labels = [
    _ for _ in cluster_labels
    if len(cluster_emb_items[_]) > 1
]
random.shuffle(labels)
for cluster_label in random.sample(labels, 5):
    show_h3(cluster_label)
    items = cluster_emb_items[cluster_label]
    random.shuffle(items)
    for item in items[:3]:
        print(item["text"])
        show_hr()

как достать ssl сетификат с сервера pki windows через powershell
для linux сервера


как достать ssl сетификат с сервера pki windows через powershell


Напиши поздравление на русском дедушку Витя/Виктор с днем рождения юбилей 70 лет в одно четверостишие


Напиши поздравление на русском дедушку Витя/Виктор с днем рождения юбилей 70 лет


Давай сыграем игру "Запор, диарея и понос". Правила игры такие: есть 100 гороха в общей тарелке. Ты можешь забрать 1, но с каждым съеденным горохом, количество гороха, которое ты можешь съесть увеличивается на количество съеденного гороха. Ты можешь выкакать какашку мне в тарелку, но твое количество гороха, которое ты можешь съесть, уменьшается до 1 гороха. Если какашка находится у меня в тарелке, я её съедаю и пропускаю ход. Игра заканчивается тогда, когда все 100 гороха были съедены и проигрывает тот, кто съел последний горох.


...

Напиши эссе на тему "почему миграции Ентов из России может привезли к мобилизации ласкутных войск?"


Напиши эссе на тему "почему миграция енотов в России может привести к убытку лакричных войск"


Сыграем в ролевую игру. Я парень, ты девушка, Мы вернулись после плавания на яхте, одетые только лишь в спасательные жилеты. Мы решили поиграть со спасательными жилетами в душе


Давай сыграем в ролевую игру. Я парень, ты девушка. На нас лишь спасательные жилеты с паховыми ремнями. После длительного и утомительного плавания мы решаем пойти в душ, чтобы поиграть с экипировкой


Давай сыграем в ролевую игру. Мне нужно надеть гидрокостюм, надувной спасательный жилет с паховыми ремнями и ремнями для ног, страховочную привязь и шлем, а затем посадить меня в лодку и пристегнуть пятиточечными ремнями безопасности в кресло, а уже в открытом море научи меня плавать. Опиши процесс застёгивания и надевания, а также мои и свои ощущения


In [83]:
dedup_emb_items = []
for cluster_label, items in cluster_emb_items.items():
    item = max(items, key=len)
    dedup_emb_items.append(item)
len(dedup_emb_items)

51319

In [ ]:
json_dump([_["id"] for _ in dedup_emb_items], "data/cluster/01_dedup_ids.json")

# prompt hardness

In [ ]:
id_emb_items = {_["id"]: _ for _ in pickle_load("data/emb.pkl")}
dedup_ids = json_load("data/cluster/01_dedup_ids.json")

In [ ]:
job_items = []
for id in dedup_ids:
    job_items.append({
        "id": id,
        "instruction": id_emb_items[id]["text"],
        "answer": None,
        "cats": None,
        "hardness": None,
        "cost": 0
    })

random.shuffle(job_items)
len(job_items)

In [ ]:
def answer_cats(answer, mapping):
    ids = (int(_) for _ in re.findall(r"\d+", answer))
    return [mapping[_] for _ in sorted(set(ids)) if _ in mapping]

_, mapping = prompt_lmsys_hardness("")

for item in job_items:
    item["cats"] = answer_cats(item["answer"], mapping)
    item["hardness"] = len(item["cats"])

In [ ]:
%run -n main.py
def answer_cats(answer, mapping):
    ids = (int(_) for _ in re.findall(r"\d+", answer))
    return [mapping[_] for _ in sorted(set(ids)) if _ in mapping]


async def worker(openrouter_client, job_item):
    messages, mapping = prompt_lmsys_hardness(job_item["instruction"])
    response = await openrouter_client(
        model="google/gemini-2.0-flash-001",
        messages=messages,
        temperature=0,
        max_tokens=50
    )
    job_item["answer"] = response["choices"][0]["message"]["content"].strip()
    job_item["cats"] = answer_cats(job_item["answer"], mapping)
    job_item["hardness"] = len(job_item["cats"])
    job_item["cost"] = response["cost"]

    logger.info(
        "[..%s] cost=%.4f len(cats)=%d",
        job_item["id"][-5:], job_item["cost"], len(job_item["cats"])
    )


async with asyncio.TaskGroup() as task_group:
    for job_item in job_items:
        if job_item["answer"]:
            continue

        task_group.create_task(worker(openrouter_client, job_item))

In [ ]:
json_dump(job_items, "data/prompt/01_hardness.json")

# prompt category

In [ ]:
id_emb_items = {_["id"]: _ for _ in pickle_load("data/emb.pkl")}
dedup_ids = json_load("data/cluster/01_dedup_ids.json")

In [ ]:
job_items = []
for id in dedup_ids:
    job_items.append({
        "id": id,
        "instruction": id_emb_items[id]["text"],
        "answer": None,
        "cat": None,
        "cost": 0
    })

random.shuffle(job_items)
len(job_items)

In [ ]:
%run -n main.py
async def worker(openrouter_client, job_item):
    messages, cats = prompt_wildchat_classify(job_item["instruction"])
    response = await openrouter_client(
        model="google/gemini-2.0-flash-001",
        messages=messages,
        temperature=0,
        max_tokens=10
    )
    job_item["answer"] = response["choices"][0]["message"]["content"].strip()

    job_item["cat"] = job_item["answer"]
    if job_item["cat"] not in cats:
        job_item["cat"] = "?"

    job_item["cost"] = response["cost"]

    logger.info(
        "[..%s] cost=%.4f cat=%s",
        job_item["id"][-5:], job_item["cost"], job_item["cat"]
    )

async with asyncio.TaskGroup() as task_group:
    for job_item in job_items:
        if job_item["answer"]:
            continue

        task_group.create_task(worker(openrouter_client, job_item))

In [ ]:
json_dump(job_items, "data/prompt/02_category.json")

In [19]:
job_items = json_load("data/prompt/02_category.json")

In [20]:
Counter(_["cat"] for _ in job_items)

Counter({'Information seeking': 16536,
         'Creative Writing': 7374,
         'Coding & Debugging': 6902,
         'Editing': 4688,
         'Math': 3044,
         'Reasoning': 2971,
         'Brainstorming': 2963,
         'Advice seeking': 2072,
         'Planning': 1601,
         'Others': 1565,
         'Role playing': 870,
         'Data Analysis': 644,
         '?': 89})

In [95]:
cat_items = defaultdict(list)
for item in job_items:
    cat_items[item["cat"]].append(item)
    
cat_counts = Counter(_["cat"] for _ in job_items)
for cat, _ in cat_counts.most_common():
    if cat != "Math":
        continue

    show_h3(cat)
    items = cat_items[cat]
    random.shuffle(items)
    for item in items[:7]:
        print(item["instruction"])
        show_hr()

Учитывая, что f(x) = 5x^ 3 - 2x + 3, найдите значение f(2).


Как я могу рассчитать количество вариаций в таблице размером 3х4?


В этой математической задаче нас просят вычислить процент от общей прибыли, полученной за каждый квартал. В первом квартале она составляет 90%, во втором - 92%, в третьем - 81% и в четвертом - 88%. Каков общий процент прибыли за определенный период времени, исходя из этих кварталов?


Предположим, у меня есть 12 яиц. Я выбрасываю 2 и съедаю 5. Сколько у меня осталось яиц?


Какова вероятность того, что при броске двух кубиков выпадет в сумме число, равное как минимум 3?


Приведите формулу для вычисления n-го члена в заданной последовательности

5, 14, 23, 32, 41, ...


Привет, мне нужно найти площадь 20-гранного кубика для домашнего задания по математике. Вы можете мне помочь с этим?


# plot hardness category

In [97]:
id_hardness = {_["id"]: _["hardness"] for _ in json_load("data/prompt/01_hardness.json")}
id_cats = {_["id"]: _["cat"] for _ in json_load("data/prompt/02_category.json")}

In [ ]:
counts = Counter()
for id in id_hardness.keys() & id_cats.keys():
    hardness = id_hardness[id]
    cat = id_cats[id]
    if cat == "?":
        continue

    counts[str(hardness), cat] += 1
    
df = pd.Series(counts)
df = df.unstack()
cats_order = df.sum(0).sort_values(ascending=False).index.to_list()
df = df.reindex(columns=cats_order)
df = df.div(df.sum(1), 0)
df = df.reset_index(names="hardness")

source = ColumnDataSource(df)
p = figure(
    x_range=df.hardness,
    height=350,
    toolbar_location=None
)
colors = Category10[10]
p.vbar_stack(
    cats_order,
    x="hardness",
    width=1,
    source=source,
    legend_label=cats_order,
    alpha=.9,
    color=Category20[len(cats_order)],
)
p.y_range.start = 0

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

p.yaxis.visible = False
p.outline_line_color = None

p.xaxis.axis_label = "hardness"

legend = p.legend[0]
legend.items = legend.items[::-1]  

p.add_layout(legend, "right")
show(p)

# umap xy

In [ ]:
id_emb_items = {_["id"]: _ for _ in pickle_load("data/emb.pkl")}
dedup_ids = json_load("data/cluster/01_dedup_ids.json")

In [ ]:
%%time
%run -n main.py
xy_items = list(umap_xy_proj([id_emb_items[_] for _ in dedup_ids]))

In [ ]:
json_dump(xy_items, "data/cluster/02_xy.json")

# plot scatter category

In [99]:
dedup_ids = json_load("data/cluster/01_dedup_ids.json")
id_emb_items = {_["id"]: _ for _ in pickle_load("data/emb.pkl")}
id_xy_items = {_["id"]: _ for _ in json_load("data/cluster/02_xy.json")}
id_cats = {_["id"]: _["cat"] for _ in json_load("data/prompt/02_category.json")}

In [ ]:
from bokeh.palettes import Category10
palette = Category10[9]

counts = Counter(id_cats.values())
cats_order = sorted(counts, key=counts.get, reverse=True)
top_cats = cats_order[:9]

data = []
for id in dedup_ids:
    emb_item = id_emb_items[id]
    xy_item = id_xy_items[id]
    cat = id_cats[id]

    color = "#dddddd"
    if cat in top_cats:
        color = palette[top_cats.index(cat) % len(palette)]


    data.append({
        "id": id,
        "x": xy_item["x"],
        "y": xy_item["y"],
        "text": emb_item["text"][:100],
        "color": color,
        "cat": cat,
    })

df = pd.DataFrame(data)
source = ColumnDataSource(df)

p = figure(
    width=800,
    height=800,
    tools=('pan, wheel_zoom, reset'),
    active_scroll='wheel_zoom'
)

p.add_tools(HoverTool(tooltips="""
<div style="width: 600px;margin-bottom:1em">
<div><b>cat:</b> @cat</div>
<div><b>text:</b> @text</div>
</div>
"""))

p.scatter(
    "x",
    "y",
    color="color",

    source=source,
    line_alpha=0,
    fill_alpha=.3,
    size=4
)

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

p.xaxis.visible = False
p.yaxis.visible = False

show(p)

In [ ]:
from bokeh.layouts import gridplot
from bokeh.palettes import Category10
palette = Category10[9]

counts = Counter(id_cats.values())
cats_order = sorted(counts, key=counts.get, reverse=True)
top_cats = cats_order[:9]

def f(cat):
    data = []
    for id in dedup_ids:
        xy_item = id_xy_items[id]

        color = "#dddddd"
        if id_cats[id] == cat:
            color = palette[top_cats.index(cat) % len(palette)]
            
        data.append({
            "id": id,
            "x": xy_item["x"],
            "y": xy_item["y"],
            "color": color,
        })

    df = pd.DataFrame(data)
    source = ColumnDataSource(df)

    p = figure(
        title=cat,
        width=250,
        height=250,
        toolbar_location=None,
    )
    p.scatter(
        "x",
        "y",
        color="color",

        source=source,
        line_alpha=0,
        fill_alpha=.15,
        size=2
    )

    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None

    p.xaxis.visible = False
    p.yaxis.visible = False

    return p


grid = [[None] * 3 for _ in range(3)]
for x in range(3):
    for y in range(3):
        cat = top_cats[y * 3 + x]
        grid[y][x] = f(cat)

show(gridplot(grid))

# cluster kmeans

In [ ]:
dedup_ids = json_load("data/cluster/01_dedup_ids.json")
id_emb_items = {_["id"]: _ for _ in pickle_load("data/emb.pkl")}

In [ ]:
%%time
cluster_input = umap_cluster_input_proj([id_emb_items[_] for _ in dedup_ids])

In [ ]:
%%time
%run -n main.py
cluster_items = list(kmeans_cluster(cluster_input, n_clusters=3000))
len(cluster_items)

In [ ]:
json_dump(cluster_items, "data/cluster/03_kmeans.json")

# select & dump

In [ ]:
cluster_items = json_load("data/cluster/03_kmeans.json")
id_emb_items = {_["id"]: _ for _ in pickle_load("data/emb.pkl")}
id_hardness = {_["id"]: _["hardness"] for _ in json_load("data/prompt/01_hardness.json")}
id_cats = {_["id"]: _["cat"] for _ in json_load("data/prompt/02_category.json")}

In [ ]:
cluster_item_ids = defaultdict(list)
for item in cluster_items:
    cluster_item_ids[item["cluster_id"]].append(item["id"])
    
cat_order_items = defaultdict(list)
cluster_order_items = defaultdict(list)
_, cats = prompt_wildchat_classify("")

for cluster_id, ids in cluster_item_ids.items():
    cluster_size = len(ids)
    cluster_hardness = median([id_hardness[_] for _ in ids])

    cat_counts = Counter(id_cats[_] for _ in ids)
    (cluster_cat, cat_size), = cat_counts.most_common(1)

    if cluster_cat == "?":
        continue
    
    cat_order_items[cluster_cat].append({
        "key": (cat_size, cluster_hardness, cluster_size),
        "cluster_id": cluster_id
    })
    
    for id in ids:
        if id_cats[id] != cluster_cat:
            continue

        cluster_order_items[cluster_id].append({
            "hardness": id_hardness[id],
            "cat": id_cats[id],
            "id": id
        })

for cat, items in cat_order_items.items():
    items.sort(key=lambda _: _["key"], reverse=True)

for cluster_id, items in cluster_order_items.items():
    items.sort(key=lambda _: _["hardness"], reverse=True)

select_items = []
while len(select_items) < 1100 and any(cat_order_items.values()):
    for cat in cats:
        if not cat_order_items[cat]:
            continue

        cat_order_item = cat_order_items[cat].pop(0)
        cluster_id = cat_order_item["cluster_id"]

        cluster_order_item = cluster_order_items[cluster_id][0]
        if cluster_order_item["hardness"] < 4:
            continue

        id = cluster_order_item["id"]
        emb_item = id_emb_items[id]
        select_items.append({
            "id": id,
            "cat": cluster_order_item["cat"],
            "hardness": cluster_order_item["hardness"],
            "instruction": emb_item["text"],
        })


In [ ]:
Counter(_["cat"] for _ in select_items)

In [ ]:
cat_hardnesses = defaultdict(list)
for item in select_items:
    cat_hardnesses[item["cat"]].append(item["hardness"])
    
for cat, hardnesses in cat_hardnesses.items():
    print(int(median(hardnesses)), cat)

In [ ]:
def is_ok(instruction):
    if re.search(r"https?://", instruction):
        return False

    if "<br/>" in instruction:
        return False
    
    return True
    
select_items = [
    _ for _ in select_items
    if is_ok(_["instruction"])
]
len(select_items)

In [ ]:
for item in select_items:
    item["instruction"] = item["instruction"].replace('\x0b', '')
    item["instruction"] = re.sub(r"^=", r"\=", item["instruction"])

In [49]:
drop_ids = [
    "16d9e519a399ee8c11b27bb76162c1e9",  # telegram token
]
select_items = [
    _ for _ in select_items
    if _["id"] not in drop_ids
]

In [50]:
json_dump(select_items, "wildchat_hard_ru.json")

In [37]:
select_items = json_load("wildchat_hard_ru.json")
for item in random.sample(select_items, 10):
    show_h3(item["id"])
    print(item["instruction"])

напиши статью 1000 слов на тему: как остановить старение организма, используй в тексте 5 раз фразу: как остановить старение организма


Расскажи пародию на "Чарли и Шоколадную фабрику", но вместо шоколадной фабрики Вилли Вонка владеет ликеро-водочным заводом. При этом: -Август чуть не утонул в реке с водкой, нахлебался и заснул пьяный до конца сказки.

-Виолетта бухнула из экспериментального вечного бочонка с вином и раздулась, как виноград. Почти как в оригинале, только теперь умпа-лумпы давят сок из неё ногами.

-Изрядно подвыпившая Верука пыталась забрать себе одну из огромных дрессированных белок-спасателей, но они, как велит им их обучение, унесли её в вытрезвитель.

-Майк по полной охренел от устройства, помещающего напитки прямо в телевизор и попытался попасть с его помощью в политическое ток-шоу и набить там всем морды. Увы, всё вышло, как в оригинале и теперь он живет в бутылке.

-Чарли получает от Вилли Водки в наследство завод и теперь его семья алкоголиков больше не будет тратить все свои деньги на спиртное, ведь они все живут на заводе. Теперь Чарли диктует цены на алкоголь по всему миру.


помоги придумать продающий пост в инстаграм про свечу ручной работы из кокосового воска с деревянным фитилем, которая называется 3 a.m. с ароматом красного вина (аккорд каберне, дополненный нотами спелой клубники и сочного красного винограда, спелой сливы и груши, нежнейших лепестков жасмина, мускуса и ванили). вызывает ассоциации с летними прогулками с подружками по ночному городу в три утра, беспечности , что скоро рассвет


сделай презентацию по теме "Путешествие по Транссибирской железной дороге". Не забудь упомянуть такие пункты, как: 1) в каких природных проложена Транссибирская магистраль и через какие крупные города она проходит. 2) расстояние между Москвой и Владивостоком и немного информации о городах. 3) сколько часовых поясов пересекают поезда Транссиба, определить разницу во времени между Москвой и Владивостоком. 4) сколько часов придётся провести в пути, если это расстояние нужно преодолеть на самолёте. 5) история создания Транссибирской магистрали, а именно время создания, цель создания, проектировщики и строители дороги. 6) самая высокая и самая западная точки транссиба, самый крутой подъём, самый длинный мост, самый большой тоннель, самый скоростной участок. 7) погода в Москве, на Байкале, во Владивостоке в разные сезоны года.


Ограничение по времени: 1 секунда
Ограничение по памяти: 256 мегабайт
Поле в игре «Речной бой» представляет собой полоску длины n клеток и шириной в одну клетку. Где‑то на поле расположен корабль из k клеток (k<=n). Какое наименьшее число выстрелов необходимо, чтобы гарантированно потопить корабль? После каждого выстрела сообщается его результат: «мимо», «ранен» или «убит».

Формат входных данных
Первая строка входных данных содержит целое число n (1<=n<=109). Вторая строка входных данных содержит целое число k (1<=k<=n).

Формат выходных данных
Выведите одно целое число — количество выстрелов.

Система оценки
Решения, правильно работающие при n<=10, будут оцениваться в 40 баллов.

Замечание
В первом примере поле состоит из n=4 клеток, корабль имеет длину k=2. Первый выстрел нужно сделать в одну из двух центральных клеток. Если результатом будет «ранен», то вторая клетка корабля находится в одной из двух соседних клеток, и за два выстрела мы гарантированно потопим корабль. Если результ

перефразировать текст Эмпирическое исследование, посвященное выявлению психологических факторов, влияющих на социально-психологическую адаптацию студентов первого курса.
 Методы, использованные в настоящем исследовании, включили в себя три основные группы: теоретические, эмпирические, методы обработки данных. В ходе теоретического анализа нами была разработана рабочая модель для проведения эмпирического исследования; опираясь на нее, мы отбирали диагностические методики для измерения факторов, влияющих на социально-психологическую адаптацию студентов. Основным методом обработки результатов исследования явился метод множественного регрессионного анализа. Регрессионная модель строилась для каждого из показателей социально-психологической адаптации, которые по очереди выступали в качестве зависимой переменной. В ходе эмпирического исследования был определен вклад каждого из факторов в оценку показателей социально-психологической адаптации, что позволило разработать структурную модель соци

Грузовик массой m = 4,0 т движется по мосту, представля-
ющем собой дугу окружности радиуса R = 80 м. Определите
силу давления грузовика на мост в верхней точке моста, если его скорость V = 72 км/ч. Сделайте чертеж с указанием сил, действующих на грузовик в этом месте моста. Ускорение свободного падения 10 м/c2


Как вести переговоры об оплате? Что делать если роботадатель не говорит об оплате


Ребенок упал с кровати и ударился головой, после чего поплакал и уснул. Может ли быть сотресение головы


Напиши описание для видеоролика на Youtube про фильм 2023 года spider man across the spider verse 2, в ролике задаются вопросы про главных героев, нужно найти различия, убрать лишнего эмоджи, угадать персонажа по эмодзи, угадать персонажа по голосу


# plot scatter kmeans + select

In [101]:
dedup_ids = json_load("data/cluster/01_dedup_ids.json")
id_emb_items = {_["id"]: _ for _ in pickle_load("data/emb.pkl")}
id_cluster_items = {_["id"]: _ for _ in json_load("data/cluster/03_kmeans.json")}
id_xy_items = {_["id"]: _ for _ in json_load("data/cluster/02_xy.json")}
select_ids = {_["id"] for _ in json_load("wildchat_hard_ru.json")}

In [ ]:
from bokeh.palettes import Category10
palette = Category10[9]

data = []
for id in dedup_ids:
    emb_item = id_emb_items[id]
    xy_item = id_xy_items[id]
    cluster_id = id_cluster_items[id]["cluster_id"]

    color = palette[cluster_id % len(palette)]
    data.append({
        "id": id,
        "x": xy_item["x"],
        "y": xy_item["y"],
        "text": emb_item["text"][:100],
        "color": color,
      })

df = pd.DataFrame(data)
source = ColumnDataSource(df)

p = figure(
    width=800,
    height=800,
    tools=('pan, wheel_zoom, reset'),
    active_scroll='wheel_zoom'
)

p.add_tools(HoverTool(tooltips="""
<div style="width: 600px;margin-bottom:1em">
<div><b>text:</b> @text</div>
</div>
"""))

p.scatter(
    "x",
    "y",
    color="color",
    source=source,
    line_alpha=0,
    fill_alpha=.1,
    size=4
)

df = df[df.id.isin(select_ids)]
source = ColumnDataSource(df)
p.scatter(
    "x",
    "y",
    color="black",
    source=source,
    line_alpha=.8,
    fill_alpha=0,
    size=4
)

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

p.xaxis.visible = False
p.yaxis.visible = False

show(p)

# prompt ttech hardness + category

In [ ]:
job_items = []
for item in mt_bench_items + arena_hard_items + alpaca_eval_items:
    job_items.append({
        "id": item["id"],
        "instruction": item["instruction"],
        "hardness_cats": None,
        "hardness": None,
        "cat": None,
        "cost": 0
    })
random.shuffle(job_items)
len(job_items)

In [ ]:
def answer_cats(answer, mapping):
    ids = (int(_) for _ in re.findall(r"\d+", answer))
    return [mapping[_] for _ in sorted(set(ids)) if _ in mapping]


async def worker(openrouter_client, job_item):
    messages, mapping = prompt_lmsys_hardness(job_item["instruction"])
    response = await openrouter_client(
        model="google/gemini-2.0-flash-001",
        messages=messages,
        temperature=0,
        max_tokens=50
    )
    answer = response["choices"][0]["message"]["content"].strip()
    job_item["hardness_cats"] = answer_cats(answer, mapping)
    job_item["hardness"] = len(job_item["hardness_cats"])
    job_item["cost"] += response["cost"]

    messages, cats = prompt_wildchat_classify(job_item["instruction"])
    response = await openrouter_client(
        model="google/gemini-2.0-flash-001",
        messages=messages,
        temperature=0,
        max_tokens=10
    )
    answer = response["choices"][0]["message"]["content"].strip()
    job_item["cat"] = answer
    if job_item["cat"] not in cats:
        job_item["cat"] = "?"
    job_item["cost"] += response["cost"]
    
    logger.info(
        "[..%s] cost=%.4f hardness=%d cat=%s",
        job_item["id"][-20:], job_item["cost"], job_item["hardness"], job_item["cat"]
    )


async with asyncio.TaskGroup() as task_group:
    for job_item in job_items:
        if job_item["cat"] is not None:
            continue

        task_group.create_task(worker(openrouter_client, job_item))

In [ ]:
json_dump(job_items, "data/prompt/03_ttech.json")

# compare ttech

In [34]:
id_cats = {}
id_hardness = {}

wildchat_hard_items = json_load("wildchat_hard_ru.json")
for item in wildchat_hard_items:
    id_cats[item["id"]] = item["cat"]
    id_hardness[item["id"]] = item["hardness"]
    
job_items = json_load("data/prompt/03_ttech.json")
for item in job_items:
    id_cats[item["id"]] = item["cat"]
    id_hardness[item["id"]] = item["hardness"]

In [44]:
%run -n main.py
compare_items = []
for source, items in [
    ("mt_bench", mt_bench_items),
    ("arena_hard", arena_hard_items),
    ("alpaca_eval", alpaca_eval_items),
    ("wildchat_hard", wildchat_hard_items),
]:
    for item in items:
        compare_items.append({
            "id": item["id"],
            "source": source,
            "instruction": item["instruction"],
            "cat": id_cats[item["id"]],
            "hardness": id_hardness[item["id"]]
        })
        
source_hardnesses = defaultdict(list)
source_cat_counts = defaultdict(Counter)
for item in compare_items:
    source_hardnesses[item["source"]].append(item["hardness"])
    source_cat_counts[item["source"]][item["cat"]] += 1
    
sources = ["alpaca_eval", "mt_bench", "arena_hard", "wildchat_hard"]
for source in sources:
    print(int(median(source_hardnesses[source])), round(mean(source_hardnesses[source]), 1), source)
print()

mapping = {
    'Creative Writing': "Writing",
    'Coding & Debugging': "Coding",
    'Advice seeking': "Advice",
    "Information seeking": "Info seeking"
}

for source in sources:
    cat_counts = source_cat_counts[source]
    total = sum(cat_counts.values())
    print(source)
    for cat, count in cat_counts.most_common(5):
        cat = mapping.get(cat, cat)
        print("%d%%&nbsp;%s<br/>" % (round(count / total * 100), cat.replace(" ", "&nbsp;")))
    print("...")
    print()

3 3.1 alpaca_eval
4 4.1 mt_bench
5 4.8 arena_hard
6 5.4 wildchat_hard

alpaca_eval
42%&nbsp;Info&nbsp;seeking<br/>
10%&nbsp;Writing<br/>
9%&nbsp;Brainstorming<br/>
8%&nbsp;Advice<br/>
7%&nbsp;Reasoning<br/>
...

mt_bench
19%&nbsp;Info&nbsp;seeking<br/>
16%&nbsp;Reasoning<br/>
15%&nbsp;Math<br/>
12%&nbsp;Coding<br/>
10%&nbsp;Role&nbsp;playing<br/>
...

arena_hard
51%&nbsp;Coding<br/>
14%&nbsp;Info&nbsp;seeking<br/>
7%&nbsp;Reasoning<br/>
6%&nbsp;Planning<br/>
5%&nbsp;Math<br/>
...

wildchat_hard
15%&nbsp;Coding<br/>
14%&nbsp;Writing<br/>
14%&nbsp;Info&nbsp;seeking<br/>
12%&nbsp;Editing<br/>
12%&nbsp;Brainstorming<br/>
...



In [48]:
for source in sources:
    if source != "mt_bench":
        continue

    items = [_ for _ in compare_items if _["source"] == source]
    random.shuffle(items)
    show_h3(source)
    for item in items[:100]:
        print("cat:", item["cat"])
        print("hardness:", item["hardness"])
        print()
        print(item["instruction"])
        show_hr()
        

cat: Editing
hardness: 2

Подготовьте профессиональное электронное письмо, в котором просите отзыв вашего руководителя о подготовленном вами "Ежеквартальном финансовом отчете". Попросите подробно рассказать об анализе данных, стиле изложения и четкости сделанных выводов. Письмо должно быть кратким и по существу.


cat: Coding & Debugging
hardness: 5

Напишите программу на C++, чтобы найти n-е число Фибоначчи с помощью рекурсии.


cat: Reasoning
hardness: 6

Внимательно прочтите приведенный ниже отрывок и развернуто ответьте на вопросы:
В небольшой компании парковочные места зарезервированы для высшего руководства: генерального директора, президента, вице-президента, секретаря и казначея, причем места расположены в указанном порядке. Охранник автостоянки может с первого взгляда определить, правильно ли припаркованы автомобили, посмотрев на их цвет. Автомобили желтые, зеленые, фиолетовые, красные и синие, а руководителей зовут Элис, Берт, Шерил, Дэвид и Энид.
* Машина на первом месте красная.
* Между красной и зеленой машинами припаркована синяя машина.
* Машина на последнем месте фиолетовая.
* Секретарша водит желтую машину.
* Машина Элис припаркована рядом с машиной Дэвида.
* Энид ездит на зеленой машине.
* Машина Берта припаркована между машинами Шерил и Энид.
* Машина Дэвида припаркована на последнем свободном месте.
Вопрос: Как зовут секретаршу?


cat: Reasoning
hardness: 1

Представьте, что вы участвуете в гонке с группой людей. Если вы только что обогнали второго участника, каково ваше текущее положение? Где находится человек, которого вы только что обогнали?


cat: Data Analysis
hardness: 5

Получив набор сложных уравнений, извлеките из каждого уравнения все уникальные имена переменных. Верните результаты в виде строки в формате JSON, выделив для каждого уравнения по одной строке.
```
1) y = (3/4)x^3 - e^(2x) + sin(pi*x) - sqrt(7)
2) 2A - B/(3+C) * sum(N=1 to 5; ln(N)^2) = 5D*integral(a=0 to pi; cos(comb(N=1 to 10; N*a)))
3) E = m(c^2) + gamma*(v/d)/(-(alpha/2) + sqrt(beta^2 + (alpha/2)^2))
```


...

cat: Data Analysis
hardness: 4

Определите именованные объекты (людей, организации, местоположения), упомянутые в данной новостной статье. Пожалуйста, создайте словарь в формате JSON, в котором перечислены именованные объекты в трех отдельных группах в зависимости от их типов объектов. Ключ - это тип объекта, а значение - это список строк.

Вчера Адамсон Эмерсон, генеральный директор Faraday, и Дитер Цетше, генеральный директор Daimler AG, объявили о планах строительства нового завода Gigafactory в Берлине. Предприятие станет совместным предприятием Faraday и Daimler, которое будет производить электромобили и аккумуляторные батареи для обеих компаний, создавая тысячи рабочих мест в регионе. Emerson и Zetsche заявили, что стратегическое расположение Берлина в сочетании с его квалифицированной рабочей силой и развитой инфраструктурой делает его идеальным местом для расширения. Цель нового завода Gigafactory - удовлетворить растущий спрос на электромобили в Европе и внести свой вклад в ус

cat: Coding & Debugging
hardness: 6

Разработайте программу на языке Python, которая считывает все текстовые файлы в директории и возвращает топ-5 слов с наибольшим количеством вхождений.


cat: Role playing
hardness: 4

Пожалуйста, примерьте на себя роль тренера по взаимоотношениям. Вам будут предоставлены подробные сведения о двух людях, оказавшихся в конфликте, и ваша задача будет заключаться в том, чтобы предложить рекомендации по решению их проблем и преодолению разрыва между ними. Это включает в себя консультирование по эффективным методам коммуникации или разработка стратегий, направленных на улучшение понимания точек зрения друг друга. Для начала я хотел бы, чтобы вы обратились со следующей просьбой: "Мне нужна помощь в разрешении конфликтов между мной и моей супругой".


cat: Information seeking
hardness: 2

Опишите пять ключевых принципов оценки аргументации в аналитическом письме.


cat: Math
hardness: 6

Какова вероятность того, что при броске двух кубиков выпадет в сумме число, равное как минимум 3?
